In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Samoded,RU,2021-12-04 12:33:50,63.6083,40.5111,4.24,83,100,11.63
1,1,Esmeraldas,BR,2021-12-04 12:33:50,-19.7625,-44.3139,75.31,68,40,8.32
2,2,Thompson,CA,2021-12-04 12:33:51,55.7435,-97.8558,-12.84,77,90,3.44
3,3,Rikitea,PF,2021-12-04 12:33:51,-23.1203,-134.9692,75.52,75,56,12.35
4,4,Ushuaia,AR,2021-12-04 12:32:16,-54.8000,-68.3000,53.22,50,0,6.91


In [3]:
# One caveat to using gmaps: The data we use for any mapping must be either an integer or a floating-point decimal number. 
# Check the data types for the columns of our DataFrame.

# Recall that you use the dtypes method to get the data types of a DataFrame.

In [4]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

# Create a Maximum Temperature Heatmap

In [5]:
# First, tell gmaps to use your API key. You only need to configure gmaps to use your API key once.

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# The general syntax for creating a heatmap is as follows.
# 1. Assign the locations to an array of latitude and longitude pairs.
# 2. Assign the weights variable to some values.
# 3. Assign the figure variable to the gmaps.figure() attribute.
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
# 5. Add the heatmap layer.
# 6. Call the figure to plot the data.

In [7]:
# Google heatmaps do not plot negative numbers. If you have a maximum temperature that is less than 0 °F, 
# then you will get an InvalidWeightException error for this line of code

# To remove the negative temperatures we can use a for loop to iterate through the max_temp and 
# add the temperatures that are greater than 0 °F to a new list.

In [8]:
# Get the maximum temperature.

# 1. Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# 2. Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# 3. Assign the figure variable.
fig = gmaps.figure()
# 4. Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# 5. Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Adjust Heatmap Zoom, Intensity, and Point Radius

In [9]:
# Map is too large. First, add the geographic center of Earth in the form of latitude and longitude (30.0° N and 31.0° E). 
# Also, add a zoom level so that only one map of Earth is shown.

In [10]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# There is a dissipation option for creating heatmaps that can be added to the gmaps.heat_layer() attribute
# The options for this are:
# 1. The default option for the dissipation is "True," so we need to set our "dissipation" to "False."
# 2. We can add max_intensity to make each measurement have a better gradient variance.
# 3. We can add point_radius to make each measurement radius larger.

In [12]:
# 1. Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# 2. Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# 3. Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 4. Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], 
                                 dissipating=False, max_intensity=300, point_radius=4)
# 5. Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Heatmap of windspeed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [30]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Esmeraldas,BR,2021-12-04 12:33:50,-19.7625,-44.3139,75.31,68,40,8.32
3,3,Rikitea,PF,2021-12-04 12:33:51,-23.1203,-134.9692,75.52,75,56,12.35
6,6,Kupang,ID,2021-12-04 12:33:51,-10.1667,123.5833,81.18,89,20,1.14
7,7,Arraial Do Cabo,BR,2021-12-04 12:33:52,-22.9661,-42.0278,80.56,61,40,8.05
8,8,Hambantota,LK,2021-12-04 12:33:52,6.1241,81.1185,80.94,94,20,3.44
12,12,Liku,ID,2021-12-04 12:33:53,-0.7275,119.8922,76.84,87,100,1.10
17,17,Hithadhoo,MV,2021-12-04 12:33:55,-0.6000,73.0833,81.57,80,81,13.87
20,20,Belmonte,BR,2021-12-04 12:33:55,-15.8631,-38.8828,82.90,78,18,5.61
21,21,Yulara,AU,2021-12-04 12:33:56,-25.2406,130.9889,87.64,25,100,20.71
22,22,Kamina,CD,2021-12-04 12:33:56,-8.7386,24.9906,81.93,56,99,6.89


In [21]:
# Determine if the preferred_cities_df DataFrame has any null values for any of the rows
preferred_cities_df.count()

City_ID       151
City          151
Country       149
Date          151
Lat           151
Lng           151
Max Temp      151
Humidity      151
Cloudiness    151
Wind Speed    151
dtype: int64

In [23]:
# Have some rows with null values, you'll need to drop them using the dropna() method 
# at the end of your filtering statement when you are creating the new DataFrame
preferred_cities2_df = preferred_cities_df.dropna()
preferred_cities2_df.count()

City_ID       149
City          149
Country       149
Date          149
Lat           149
Lng           149
Max Temp      149
Humidity      149
Cloudiness    149
Wind Speed    149
dtype: int64

## Get Travel Destinations

In [31]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Esmeraldas,BR,75.31,-19.7625,-44.3139,
3,Rikitea,PF,75.52,-23.1203,-134.9692,
6,Kupang,ID,81.18,-10.1667,123.5833,
7,Arraial Do Cabo,BR,80.56,-22.9661,-42.0278,
8,Hambantota,LK,80.94,6.1241,81.1185,
12,Liku,ID,76.84,-0.7275,119.8922,
17,Hithadhoo,MV,81.57,-0.6000,73.0833,
20,Belmonte,BR,82.90,-15.8631,-38.8828,
21,Yulara,AU,87.64,-25.2406,130.9889,
22,Kamina,CD,81.93,-8.7386,24.9906,


In [32]:
# Using the latitude and longitude and specific parameters, use the Google Places Nearby Search request to retrieve a hotel 
# and add it to the Hotel Name column.

In [36]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Esmeraldas,BR,75.31,-19.7625,-44.3139,Bom Será - Marcinho e Fernanda
3,Rikitea,PF,75.52,-23.1203,-134.9692,People ThankYou
6,Kupang,ID,81.18,-10.1667,123.5833,Hotel On The Rock by Prasanthi
7,Arraial Do Cabo,BR,80.56,-22.9661,-42.0278,Pousada Porto Praia
8,Hambantota,LK,80.94,6.1241,81.1185,Bungalow 63
12,Liku,ID,76.84,-0.7275,119.8922,Wahyu
17,Hithadhoo,MV,81.57,-0.6000,73.0833,Scoop Guest House
20,Belmonte,BR,82.90,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
21,Yulara,AU,87.64,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
22,Kamina,CD,81.93,-8.7386,24.9906,Guest House Florvia


## Create a Maximum Temperature Heatmap from a Hotel DataFrame

In [37]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
# If we refer to the gmaps documentation on how to add markers
# the syntax is markers = gmaps.marker_layer(marker_locations), where the marker_locations are latitudes and longitudes.

# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [46]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [48]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))